# Capstone Project Assignment - Part 1


Importing relevant libraries

In [5]:
import pandas as pd
import numpy as np
import folium
import geopy
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline 
from sklearn.cluster import KMeans 

In [4]:
pip install geocoder

     |████████████████████████████████| 98 kB 8.5 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


### Download data from wiki and assign to pandas dataframe using Pandas read_html function

In [6]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Shape of Actual Dataframe

In [7]:
print("Actual shape of Frame",df.shape)

Actual shape of Frame (180, 3)


### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned

In [8]:
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace = True) 

print("Shape of Frame after filtering ",df.shape)

Shape of Frame after filtering  (103, 3)


### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma
#### -- Dataframe Already have the neighbour combined per postal code

In [9]:
df[df["Postal Code"]=="M5A"]

,Postal Code,Borough,Neighbourhood
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Check if any Postal Code have duplicate entry in dataframe

In [10]:
df['Postal Code'].duplicated().any()

False

### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [11]:
df["Neighbourhood"].loc[df["Neighbourhood"]=="Not assigned"] = df["Borough"]

### Checking if any row exists with Neighbourhood = "Not assigned"

In [12]:
df[df["Neighbourhood"]=="Not assigned"].count()

Postal Code      0
Borough          0
Neighbourhood    0
dtype: int64

### Final shape of Dataframe after Cleanup

In [13]:
print("Final shape",df.shape)

Final shape (103, 3)


### Reset Index of Dataframe after filtering


In [14]:
df.reset_index(inplace=True, drop=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
